In [1]:
import pickle as pkl
import os
import pandas as pd

path = os.getcwd()
data_name = 'SMD'
data_file = os.path.join(path, 'processed', data_name)
train_data_ = open(os.path.join(data_file, 'machine-1-1_train.pkl'),'rb')
train_data = pd.DataFrame(pkl.load(train_data_))
test_data_ = open(os.path.join(data_file, 'machine-1-1_test.pkl'),'rb')
test_data = pd.DataFrame(pkl.load(test_data_))

In [2]:
test_data_label_ = open(os.path.join(data_file, 'machine-1-1_test_label.pkl'),'rb')
test_data_label = pd.DataFrame(pkl.load(test_data_label_),columns=['label'])

In [3]:
anomaly_num = len(test_data_label[test_data_label.label==1])
anomaly_num / len(test_data_label)

0.09459601811861372

In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_train_data = pd.DataFrame(scaler.fit_transform(train_data))
normalized_test_data = pd.DataFrame(scaler.fit_transform(test_data))
normalized_test_data

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.065217,0.065678,0.068156,0.073256,0.0,0.925287,0.269886,0.0,0.031081,0.000000,...,0.0,0.004317,0.068036,0.048893,0.000386,0.000023,0.064432,0.064500,0.0,0.0
1,0.076087,0.080508,0.073742,0.075582,0.0,0.922414,0.270833,0.0,0.031081,0.000122,...,0.0,0.004317,0.070020,0.050437,0.000386,0.000011,0.065228,0.065224,0.0,0.0
2,0.065217,0.064619,0.069273,0.073256,0.0,0.919540,0.270833,0.0,0.030940,0.000366,...,0.0,0.004317,0.069684,0.055069,0.000386,0.000034,0.067111,0.067178,0.0,0.0
3,0.076087,0.048729,0.061452,0.069768,0.0,0.919540,0.268940,0.0,0.027250,0.000244,...,0.0,0.005756,0.073253,0.051467,0.000000,0.000023,0.066676,0.066744,0.0,0.0
4,0.076087,0.051907,0.060335,0.069768,0.0,0.925287,0.269886,0.0,0.030940,0.000244,...,0.0,0.004317,0.070932,0.051467,0.000386,0.000011,0.066604,0.066671,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.065217,0.051907,0.045810,0.046512,0.0,0.896551,0.252841,0.0,0.043571,0.000244,...,0.0,0.027339,0.046733,0.040144,0.000000,0.000011,0.042931,0.043000,0.0,0.0
28475,0.054347,0.025424,0.036871,0.043024,0.0,0.893678,0.252841,0.0,0.032501,0.000000,...,0.0,0.027339,0.047438,0.048893,0.000000,0.000045,0.046550,0.046619,0.0,0.0
28476,0.054347,0.080508,0.048044,0.046512,0.0,0.896551,0.253788,0.0,0.026114,0.000611,...,0.0,0.027339,0.046797,0.040144,0.000386,0.000034,0.043003,0.043000,0.0,0.0
28477,0.054347,0.056144,0.045810,0.045350,0.0,0.890805,0.252841,0.0,0.033210,0.000122,...,0.0,0.027339,0.041884,0.043232,0.000000,0.000034,0.039890,0.039959,0.0,0.0


In [5]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
pca_train_data = pca.fit_transform(normalized_train_data)
pca_test_data = pca.fit_transform(normalized_test_data)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.64724463 0.23495746 0.03814478 0.01958586 0.01584371]
[67.198586 40.487457 16.31336  11.689536 10.513671]


## Ensemble

In [6]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import metrics

import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.combination import aom, moa, average, maximization, majority_vote
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

In [7]:
def pred_labels(y_test, test_scores):
    fpr, tpr, thresholds = roc_curve(y_test, test_scores)
    cutoff = thresholds[np.argmax(tpr - fpr)]
    pred_label = []
    for each in test_scores:
        if each > cutoff:
            pred_label.append(1)
        else:
            pred_label.append(0)    
    return pred_label

In [8]:
from sklearn.metrics.cluster import normalized_mutual_info_score
def MI_weights(test_scores):
    mi_matrix = np.zeros(shape=(4,4))
    for i in range(4):
        for j in range(4):
            base_a = test_scores[:,i]
            base_b = test_scores[:,j]
            mi_matrix[i][j] = normalized_mutual_info_score(base_a, base_b)
    #mi_avg = 1/((sum(mi_matrix) - 1)/4)
    mi_avg = (sum(mi_matrix) - 1)/3
    weights = mi_avg/sum(mi_avg)
    estimator_weights = weights.reshape(1,4)
    return estimator_weights

In [9]:
def cal_eval(label, pred_label, scores):
    pr = round(metrics.precision_score(label, pred_label, average='macro'), ndigits=4)
    re = round(metrics.recall_score(label, pred_label, average='macro'), ndigits=4)
    f1 = round(metrics.f1_score(label, pred_label, average='macro'), ndigits=4)
    roc = round(roc_auc_score(label, scores), ndigits=4)
    return pr,re,f1,roc

In [10]:
X_train = pca_test_data
X_test = pca_test_data
y_train = test_data_label
y_test = test_data_label

In [11]:
#Ensemble methods
def ensemble_models():
    df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
    enm_avg_df = pd.DataFrame(columns=df_columns)
    enm_max_df = pd.DataFrame(columns=df_columns)
    enm_w_avg_df = pd.DataFrame(columns=df_columns)
    
    outliers_fraction = np.count_nonzero(y_train) / len(y_train)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    random_state = np.random.RandomState(42)
    classifiers = {
            'Isolation Forest': IForest(contamination=outliers_fraction,
                                        random_state=random_state),
            'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
            'Local Outlier Factor (LOF)': LOF(
                contamination=outliers_fraction),
            'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction)
        }

    n_clf = len(classifiers)
    train_scores = np.zeros([X_train.shape[0], n_clf])
    test_scores = np.zeros([X_test.shape[0], n_clf])
    p_labels = np.zeros([X_test.shape[0], n_clf])

    i = 0
    train_duration = 0
    test_duration = 0
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train)
        t1 = time()
        test_sccore = clf.decision_function(X_test)
        t_t = time()
        test_mean_score = np.nanmean(test_scores)
        test_scores[np.isnan(test_scores)] = test_mean_score
        test_scores[:, i] = test_sccore
        p_labels[:, i] = pred_labels(y_test, test_sccore)
        i += 1
        print('Base detector %i is fitted for prediction' % i)
        train_duration += round(t1 - t0, ndigits=4)
        test_duration += round(t_t - t1, ndigits=4)

    # standardize test score
    test_scores_norm = standardizer(test_scores)

    # Combination by average
    t2 = time()
    y_by_average = average(test_scores_norm)
    t3 = time()
    evaluate_print('Combination by Average', y_test, y_by_average)
    pred_label = pred_labels(y_test, y_by_average)
    pr_avg, re_avg, f1_avg, roc_avg = cal_eval(y_test, pred_label, y_by_average)
    train_time_avg = round(train_duration+t3-t2, ndigits=4)
    test_time_avg = round(test_duration+t3-t2, ndigits=4)

    enm_avg = pd.DataFrame([pr_avg, re_avg, f1_avg, roc_avg, train_time_avg, test_time_avg]).transpose()
    enm_avg.columns = df_columns
    enm_avg_df = pd.concat([enm_avg_df, enm_avg], axis=0)

    # Combination by max
    t4 = time()
    y_by_maximization = maximization(test_scores_norm)
    t5 = time()
    evaluate_print('Combination by Maximization', y_test, y_by_maximization)
    pred_label = pred_labels(y_test, y_by_maximization)
    pr_max, re_max, f1_max, roc_max = cal_eval(y_test, pred_label, y_by_maximization)
    train_time_max = round(train_duration+t5-t4, ndigits=4)
    test_time_max = round(test_duration+t5-t4, ndigits=4)

    enm_max = pd.DataFrame([pr_max, re_max, f1_max, roc_max, train_time_max, test_time_max]).transpose()
    enm_max.columns = df_columns
    enm_max_df = pd.concat([enm_max_df, enm_max], axis=0)

    # Combination by weighted average
    t6 = time()
    pred_label_0 = pred_labels(y_test, test_scores_norm[:,0])
    pred_label_1 = pred_labels(y_test, test_scores_norm[:,1]) 
    pred_label_2 = pred_labels(y_test, test_scores_norm[:,2]) 
    pred_label_3 = pred_labels(y_test, test_scores_norm[:,3]) 
    pred_label_ = np.array((pred_label_0, pred_label_1, pred_label_2, pred_label_3)).T
    estimator_weights = MI_weights(pred_label_)
    #estimator_weights = MI_weights(test_scores_norm)
    y_by_w_average = average(test_scores_norm, estimator_weights)
    t7 = time()
    evaluate_print('Combination by weighted Average', y_test, y_by_w_average)
    pred_label = pred_labels(y_test, y_by_w_average)
    pr_w_avg, re_w_avg, f1_w_avg, roc_w_avg = cal_eval(y_test, pred_label, y_by_w_average)
    train_time_w_avg = round(train_duration+t7-t6, ndigits=4)
    test_time_w_avg = round(test_duration+t7-t6, ndigits=4)

    enm_w_avg = pd.DataFrame([pr_w_avg, re_w_avg, f1_w_avg, roc_w_avg, train_time_w_avg, test_time_w_avg]).transpose()
    enm_w_avg.columns = df_columns
    enm_w_avg_df = pd.concat([enm_w_avg_df, enm_w_avg], axis=0)
    
    return enm_avg_df, enm_max_df, enm_w_avg_df

In [12]:
df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']

t_enm_avg_df = pd.DataFrame(columns=df_columns)
t_enm_max_df = pd.DataFrame(columns=df_columns)
t_enm_w_avg_df = pd.DataFrame(columns=df_columns)

for i in range(10):
    print(i)
    enm_avg, enm_max, env_w_avg = ensemble_models()
    t_enm_avg_df = pd.concat([t_enm_avg_df, enm_avg], axis=0, ignore_index=True)
    t_enm_max_df = pd.concat([t_enm_max_df, enm_max], axis=0, ignore_index=True)
    t_enm_w_avg_df = pd.concat([t_enm_w_avg_df, env_w_avg], axis=0, ignore_index=True)

0
Base detector 1 is fitted for prediction
Base detector 2 is fitted for prediction
Base detector 3 is fitted for prediction
Base detector 4 is fitted for prediction
Combination by Average ROC:0.8839, precision @ rank n:0.435
Combination by Maximization ROC:0.8864, precision @ rank n:0.4543
Combination by weighted Average ROC:0.8945, precision @ rank n:0.4699
1
Base detector 1 is fitted for prediction
Base detector 2 is fitted for prediction
Base detector 3 is fitted for prediction
Base detector 4 is fitted for prediction
Combination by Average ROC:0.8839, precision @ rank n:0.435
Combination by Maximization ROC:0.8864, precision @ rank n:0.4543
Combination by weighted Average ROC:0.8945, precision @ rank n:0.4699
2
Base detector 1 is fitted for prediction
Base detector 2 is fitted for prediction
Base detector 3 is fitted for prediction
Base detector 4 is fitted for prediction
Combination by Average ROC:0.8839, precision @ rank n:0.435
Combination by Maximization ROC:0.8864, precision 

In [13]:
print('Ensemble average')
metrics = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
mean_avg = []
std_avg = []
for each in metrics:
    mean_avg.append(round(t_enm_avg_df[each].mean(), ndigits=4))
    std_avg.append(round(t_enm_avg_df[each].std(), ndigits=4))
print(mean_avg)
print(std_avg)
t_enm_avg_df

Ensemble average
[0.6455, 0.8278, 0.6683, 0.8839, 68.525, 36.3232]
[0.0, 0.0, 0.0, 0.0, 0.9557, 0.346]


,Precision,Recall,F1 score,AUC,Train time,Test time
0,0.6455,0.8278,0.6683,0.8839,68.1963,36.4011
1,0.6455,0.8278,0.6683,0.8839,68.8871,35.8695
2,0.6455,0.8278,0.6683,0.8839,68.4448,36.3864
3,0.6455,0.8278,0.6683,0.8839,67.9542,36.1347
4,0.6455,0.8278,0.6683,0.8839,67.9014,36.0816
5,0.6455,0.8278,0.6683,0.8839,67.8065,36.0439
6,0.6455,0.8278,0.6683,0.8839,68.1116,36.4677
7,0.6455,0.8278,0.6683,0.8839,69.5228,36.8764
8,0.6455,0.8278,0.6683,0.8839,67.6905,36.0916
9,0.6455,0.8278,0.6683,0.8839,70.7344,36.8792


In [14]:
print('Ensemble max')
metrics = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
mean_max = []
std_max = []
for each in metrics:
    mean_max.append(round(t_enm_max_df[each].mean(), ndigits=4))
    std_max.append(round(t_enm_max_df[each].std(), ndigits=4))
print(mean_max)
print(std_max)
t_enm_max_df

Ensemble max
[0.6675, 0.8405, 0.7018, 0.8864, 68.5253, 36.3235]
[0.0, 0.0, 0.0, 0.0, 0.9557, 0.346]


,Precision,Recall,F1 score,AUC,Train time,Test time
0,0.6675,0.8405,0.7018,0.8864,68.1966,36.4014
1,0.6675,0.8405,0.7018,0.8864,68.8873,35.8697
2,0.6675,0.8405,0.7018,0.8864,68.4451,36.3867
3,0.6675,0.8405,0.7018,0.8864,67.9547,36.1352
4,0.6675,0.8405,0.7018,0.8864,67.9017,36.0819
5,0.6675,0.8405,0.7018,0.8864,67.8069,36.0443
6,0.6675,0.8405,0.7018,0.8864,68.1119,36.4680
7,0.6675,0.8405,0.7018,0.8864,69.5232,36.8768
8,0.6675,0.8405,0.7018,0.8864,67.6908,36.0919
9,0.6675,0.8405,0.7018,0.8864,70.7347,36.8795


In [15]:
print('Ensemble weighted avg')
metrics = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
mean_w_avg = []
std_w_avg = []
for each in metrics:
    mean_w_avg.append(round(t_enm_w_avg_df[each].mean(), ndigits=4))
    std_w_avg.append(round(t_enm_w_avg_df[each].std(), ndigits=4))
print(mean_w_avg)
print(std_w_avg)
t_enm_w_avg_df

Ensemble weighted avg
[0.6578, 0.8369, 0.6876, 0.8945, 68.6471, 36.4453]
[0.0, 0.0, 0.0, 0.0, 0.957, 0.3484]


,Precision,Recall,F1 score,AUC,Train time,Test time
0,0.6578,0.8369,0.6876,0.8945,68.3295,36.5343
1,0.6578,0.8369,0.6876,0.8945,69.0034,35.9858
2,0.6578,0.8369,0.6876,0.8945,68.5639,36.5055
3,0.6578,0.8369,0.6876,0.8945,68.0727,36.2532
4,0.6578,0.8369,0.6876,0.8945,68.0209,36.2011
5,0.6578,0.8369,0.6876,0.8945,67.9284,36.1658
6,0.6578,0.8369,0.6876,0.8945,68.2302,36.5863
7,0.6578,0.8369,0.6876,0.8945,69.6461,36.9997
8,0.6578,0.8369,0.6876,0.8945,67.8142,36.2153
9,0.6578,0.8369,0.6876,0.8945,70.8615,37.0063
